In [ ]:
import pandas as pd
from pathlib import Path
from datetime import datetime
from collections import Counter
import re
import string
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
# Basic paths

# If needed in a fresh environment, run once:
nltk.download("vader_lexicon")

# ---------- Paths ----------
base_path = Path("/Users/apple/Desktop/30112_python/Scrape_Reddit")
cleaned_dir = base_path / "scraped_data"

in_path = cleaned_dir / "reddit_inflation_2020_2025_posts_and_comments_TEST_clean.parquet"

print("Loading:", in_path)
df = pd.read_parquet(in_path)
print("Loaded shape:", df.shape)
print("\nColumns:", df.columns.tolist())

In [ ]:
# DIAGNOSTIC: Check datetime format
print("\n=== DATETIME DIAGNOSTIC ===")
print("created_utc dtype:", df["created_utc"].dtype)
print("\nFirst 10 timestamps:")
print(df["created_utc"].head(10))

# Check if year column exists and what it contains
if "year" in df.columns:
    print("\nYear distribution (existing):")
    print(df["year"].value_counts().sort_index())
else:
    print("\nNo 'year' column found - will be created from created_utc")

# FIXED: Ensure created_utc is properly formatted as datetime
if not pd.api.types.is_datetime64_any_dtype(df["created_utc"]):
    print("\nWARNING: created_utc is not datetime type. Converting...")
    # Try to convert - if it's already timestamps, use unit='s'
    if pd.api.types.is_numeric_dtype(df["created_utc"]):
        df["created_utc"] = pd.to_datetime(df["created_utc"], unit="s", utc=True)
    else:
        df["created_utc"] = pd.to_datetime(df["created_utc"], utc=True, errors="coerce")
    print("Conversion complete. New dtype:", df["created_utc"].dtype)
    print("Sample converted values:")
    print(df["created_utc"].head())

print("\n=== END DIAGNOSTIC ===")

In [ ]:
# Create Phase Labels
# Updated with more realistic inflation phases based on actual timeline

def assign_inflation_phase(dt: datetime) -> str:
    """
    Assign inflation phase based on actual US inflation timeline:
    - Pre-inflation: 2020-2021 (low inflation, ~1-2%)
    - Rising: Jan 2021 - Dec 2021 (inflation starts climbing, 1% -> 7%)
    - Peak: 2022 - mid 2023 (high inflation, peaked at 9.1% June 2022)
    - Cooling: Mid 2023 - 2025 (inflation declining, ~3-4%)
    """
    if pd.isna(dt):
        return "unknown"

    year = dt.year
    month = dt.month
    
    # Pre-inflation period (stable low inflation)
    if year == 2020:
        return "pre_inflation"
    
    # Rising inflation period (starting to climb)
    elif year == 2021:
        return "rising"
    
    # Peak inflation period
    elif year == 2022:
        return "peak"
    
    # Cooling period (inflation declining but still elevated)
    elif year == 2023:
        if month <= 6:
            return "peak"  # First half of 2023 still elevated
        else:
            return "cooling"
    
    # 2024-2025: continued cooling
    elif year >= 2024:
        return "cooling"
    
    return "unknown"

# Apply phase assignment
df["inflation_phase"] = df["created_utc"].apply(assign_inflation_phase)

print("\nInflation phase counts:")
print(df["inflation_phase"].value_counts().sort_index())

# Show date range for each phase
print("\nDate range by phase:")
for phase in df["inflation_phase"].unique():
    phase_df = df[df["inflation_phase"] == phase]
    if len(phase_df) > 0:
        min_date = phase_df["created_utc"].min()
        max_date = phase_df["created_utc"].max()
        print(f"{phase}: {min_date.date()} to {max_date.date()} ({len(phase_df):,} posts/comments)")

In [ ]:
# Keyword List and helper for is_inflation

KEYWORDS = [
    # Core inflation/econ
    "inflation",
    "cost of living",
    "cost-of-living",
    "high cost of living",
    "living costs",
    "price increases",
    "rising prices",
    "cpi",
    "consumer price index",
    "interest rates",
    "mortgage rates",
    "fed",
    "federal reserve",
    "rate hike",
    "rate hikes",
    "rate increase",
    "rate increases",
    # Everyday expenses
    "gas prices",
    "gas price",
    "grocery prices",
    "grocery bill",
    "food prices",
    "food bill",
    "rent increase",
    "rent hike",
    "higher rent",
    "rent is too high",
    "housing costs",
    "housing affordability",
    "property taxes",
    "electric bill",
    "electricity bill",
    "energy bill",
    "heating bill",
    "gas bill",
    "utility bills",
    "utilities",
    # Income / strain
    "wage stagnation",
    "wages not keeping up",
    "real wages",
    "paycheck to paycheck",
    "making ends meet",
    "can't afford",
    "cannot afford",
]
KEYWORDS_LOWER = [k.lower() for k in KEYWORDS]

def text_matches_keywords(text: str) -> bool:
    if not isinstance(text, str):
        return False
    t = text.lower()
    return any(k in t for k in KEYWORDS_LOWER)

In [ ]:
# Build raw_text (fallback) and recompute is_inflation

# raw_text should already exist from the cleaning step (title + body).
# If not, create it here.
if "raw_text" not in df.columns:
    df["raw_text"] = (
        df[["title", "body"]]
        .fillna("")
        .agg(" ".join, axis=1)
        .str.strip()
    )

df["is_inflation"] = df["raw_text"].apply(text_matches_keywords)

print("is_inflation value counts:")
print(df["is_inflation"].value_counts(dropna=False))

# Manual quality check: print a few examples
print("\nSample inflation-related rows:")
if df["is_inflation"].any():
    sample_infl = df[df["is_inflation"]].sample(
        n=min(5, df["is_inflation"].sum()), random_state=42
    )
    for _, row in sample_infl.iterrows():
        print("----")
        print(row["created_utc"], "|", row["subreddit"], "|", row["type"])
        print(row["raw_text"][:400], "...\n")
else:
    print("No rows flagged as inflation-related in this TEST dataset.")

In [ ]:
# Pilot analysis (on TEST data)

df_inf = df[df["is_inflation"]].copy()
print("\nInflation-related rows:", df_inf.shape[0])
print("Date range:", df_inf["created_utc"].min(), "to", df_inf["created_utc"].max())

In [ ]:
if not df_inf.empty:
    # Group inflation-related rows by month (period 'M') and post type,
    # count how many rows fall into each (month, type) combo.
    monthly_counts = (
        df_inf
        .groupby([df_inf["created_utc"].dt.to_period("M"), "type"])
        .size()                      # number of rows in each group
        .unstack(fill_value=0)       # turn 'type' into columns
        .rename_axis(index="month")  # label the index as 'month'
    )

    print("\nMonthly counts (first 20 rows):")
    print(monthly_counts.head(20))
    
    print("\nMonthly counts (last 20 rows):")
    print(monthly_counts.tail(20))

    # Group by phase and type to see how many posts/comments per phase.
    phase_counts = (
        df_inf
        .groupby(["inflation_phase", "type"])
        .size()
        .unstack(fill_value=0)
    )

    print("\nCounts by inflation phase and type:")
    print(phase_counts)
else:
    print("No inflation-related rows; skipping counts.")

In [ ]:
if not df_inf.empty:
    print("\nTop tokens per phase:")

    # If tokens column isn't there, we can't compute token frequencies.
    if "tokens" not in df_inf.columns:
        print("No 'tokens' column found; skipping token frequency by phase.")
    else:
        # Loop over each phase.
        for phase in sorted(df_inf["inflation_phase"].unique()):
            # Keep only rows in this phase.
            phase_df = df_inf[df_inf["inflation_phase"] == phase]

            # Flatten all token lists into one long list of tokens.
            all_tokens = [
                t
                for toks in phase_df["tokens"]
                if isinstance(toks, (list, tuple))   # skip bad entries
                for t in toks
            ]

            # Count token frequencies for this phase.
            counts = Counter(all_tokens)
            print(f"\nPhase: {phase} ({len(phase_df):,} posts/comments)")
            print(counts.most_common(15))  # top 15 tokens

In [ ]:
if not df_inf.empty:
    # If clean_text isn't present, create a simple cleaned version of raw_text.
    if "clean_text" not in df_inf.columns:
        def clean_text_basic(text: str) -> str:
            if not isinstance(text, str):
                return ""
            # Remove URLs.
            text = re.sub(r"http\S+|www\.\S+", " ", text)
            # Lowercase.
            text = text.lower()
            # Remove punctuation.
            text = text.translate(str.maketrans("", "", string.punctuation))
            # Collapse multiple spaces.
            text = re.sub(r"\s+", " ", text).strip()
            return text

        df_inf["clean_text"] = df_inf["raw_text"].apply(clean_text_basic)

    # Initialize VADER sentiment analyzer (gives ± sentiment scores).
    sia = SentimentIntensityAnalyzer()

    def vader_compound(text: str) -> float:
        # Return compound score in [-1, 1]; 0 for empty/non-string.
        if not isinstance(text, str) or not text.strip():
            return 0.0
        return sia.polarity_scores(text)["compound"]

    # Compute a sentiment score for each row based on clean_text.
    df_inf["sentiment"] = df_inf["clean_text"].apply(vader_compound)

    # Average sentiment for each phase.
    sent_by_phase = df_inf.groupby("inflation_phase")["sentiment"].mean().sort_index()
    print("\nAverage sentiment by phase:")
    print(sent_by_phase)

    # Average sentiment by month (using month period of created_utc).
    sent_by_month = (
        df_inf
        .groupby(df_inf["created_utc"].dt.to_period("M"))["sentiment"]
        .mean()
        .rename_axis(index="month")
    )
    print("\nAverage sentiment by month (first 20 rows):")
    print(sent_by_month.head(20))
    print("\nAverage sentiment by month (last 20 rows):")
    print(sent_by_month.tail(20))

In [ ]:
# Save updated TEST file with new columns

out_path = cleaned_dir / "reddit_test_with_phase.parquet"

df.to_parquet(out_path, engine="pyarrow", index=False)

print("\nSaved updated TEST data with inflation_phase/is_inflation to:", out_path)